# REDES NEURAIS ARTIFICIAIS

# PARTE 1: Classificação Binária - Diagnóstico de Câncer de Mama

Base de dados: Breast Cancer Wisconsin (Diagnostic) -
https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic

In [30]:
#bilbiotecas keras e tensorflow
#!pip install scikeras==0.13.0

In [31]:
#!pip install tensorflow==2.16.1 scikit-learn==1.5.0

In [32]:
# após instalar as bibliotecas, é necessário reiniciar o notebook
# Para isso, rode esta célula sozinha antes de prosseguir.
#exit()

In [33]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

In [34]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.3.3', '2.16.1', '1.5.0', '0.13.0')

### Carregar os dados

1) Carregue a base de dados, faça a divisão de treino e teste (para isso, utilize a função train_test_split do sklearn), como o tamanho da base de teste de 0.25.

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Carregar os dados
X = pd.read_csv('dados_breast.csv').values
y = pd.read_csv('rotulos_breast.csv', header=None).values.ravel()

# Garantir que X e y tenham o mesmo tamanho (ajustar y para o tamanho de X)
min_size = min(len(X), len(y))
X = X[:min_size]
y = y[:min_size]

# inicialmente os arquivos tinham uma diferença de tamanho (um era maior que o outro)
print(f"Tamanho de X: {X.shape}")
print(f"Tamanho de y: {y.shape}")

# Dividir em treino e teste (75% treino, 25% teste)
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)

Tamanho de X: (569, 30)
Tamanho de y: (569,)


In [36]:
X_treinamento.shape, y_treinamento.shape

((426, 30), (426,))

In [37]:
X_teste.shape, y_teste.shape

((143, 30), (143,))

### Estrutura da Rede Neural Artificial e Teste

In [38]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import backend as k

2) Crie a RNA com as seguintes configurações:

a) uma camada de entrada de com 30 neurônios;

b) uma camada oculta densa com 16 neurônios. Porque a camada oculta tem essa quantidade de neurônios?

c) adicione na camada oculta a função de ativação relu e inicialize os pesos de utilize o Random uniform initializer (https://www.tensorflow.org/api_docs/python/tf/keras/initializers);

d) adicione a camada de saíde com a função de ativação sigmoid. Porque foi utilizada esta função para a saída da RNA?

3) Porque utilizamos a classe chamada Sequential para a RNA?

In [39]:
# Criar a Rede Neural Artificial
rede_neural = Sequential([
    # a) Camada de entrada com 30 neurônios (30 features do dataset)
    tf.keras.layers.InputLayer(shape=(30,)),

    # b) e c) Camada oculta densa com 16 neurônios, ativação relu e inicializador random_uniform
    tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),

    # d) Camada de saída com 1 neurônio e ativação sigmoid (classificação binária)
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

**Respostas:**

**b)** A camada oculta tem 16 neurônios como uma escolha heurística que equilibra capacidade de aprendizado e complexidade. Geralmente, usa-se um número entre a quantidade de entradas (30) e saídas (1). O valor 16 é suficiente para capturar padrões nos dados sem causar overfitting excessivo.

**d)** A função sigmoid foi utilizada na saída porque estamos fazendo classificação binária (0 ou 1: benigno ou maligno). A sigmoid transforma qualquer valor em um número entre 0 e 1, que pode ser interpretado como probabilidade da amostra pertencer à classe positiva.

**3)** Utilizamos a classe Sequential porque ela permite criar modelos de forma linear e sequencial, onde as camadas são empilhadas uma após a outra. É ideal para redes neurais feedforward simples onde cada camada tem exatamente uma entrada e uma saída.

4) A partir da RNA gerada, explique o que são os valores apresentados na tabela da rede_neural.summary()

In [40]:
rede_neural.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 513 (2.00 KB)

 Trainable params: 513 (2.00 KB)

 Non-trainable params: 0 (0.00 B)

**Resposta 4:** A tabela do summary() mostra:
- **Layer (type)**: O tipo de cada camada (InputLayer, Dense)
- **Output Shape**: Formato da saída de cada camada (None, número_neurônios), onde None representa o batch size
- **Param #**: Número de parâmetros treináveis em cada camada. Para camadas Dense, é calculado como: (neurônios_entrada × neurônios_saída) + neurônios_saída (bias)
  - Primeira camada oculta: (30 × 16) + 16 = 496 parâmetros
  - Camada de saída: (16 × 1) + 1 = 17 parâmetros
  - **Total**: 513 parâmetros treináveis

5) Adicione um otimizador Adam (https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) e especifique a classe loss binário - binary crossentropy (https://www.tensorflow.org/api_docs/python/tf/keras/losses) e a classe metrics para utilizar a métrica de avaliação de acurácia binária (https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

6) Para que servem os otimizadores? Como o otimizador Adam funciona?

7) Depois de estruturados os parâmtros da RNA, utilize a função .fit para fazer o treinamento da rede. Como foi utilizado o otimizador Adam, e ele é baseado na descida do gradiente estocástica, é possível definir a quantidade de registros que serão enviados para a RNA, isto é, em cada batch serão utilizados 10 registros. Quantos batches serão utilizados ao total?

8) Por fim, defina o número de épocas em que ocorre o treinamento igual a 100.

In [41]:
# 5) Compilar a rede neural com otimizador Adam, loss binary_crossentropy e métrica binary_accuracy
rede_neural.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

**Respostas:**

**6)** Os otimizadores servem para ajustar os pesos da rede neural durante o treinamento, minimizando a função de perda. O Adam (Adaptive Moment Estimation) combina as vantagens de dois outros otimizadores: RMSprop e Momentum. Ele adapta a taxa de aprendizado individualmente para cada parâmetro, usando médias móveis dos gradientes e dos quadrados dos gradientes, tornando-o eficiente e convergindo rapidamente.

**7)** Com batch_size=10 e 426 amostras de treino (75% de 569), teremos aproximadamente 43 batches por época (426 ÷ 10 = 42.6, arredondado para 43).

In [42]:
# 7) e 8) Treinar a rede neural com batch_size=10 e epochs=100
rede_neural.fit(X_treinamento, y_treinamento, batch_size=10, epochs=100)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - binary_accuracy: 0.5258 - loss: 11.2638   
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5329 - loss: 4.9559 
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5305 - loss: 4.4532
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5775 - loss: 2.5422
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5869 - loss: 2.3673
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5869 - loss: 2.8424  
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6127 - loss: 2.3681 
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5775 - loss: 2.6891 
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5540 - loss: 2.1462 
Epoch 10/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5775 - loss: 2.2544 
Epoch 11/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accurac

9) Crie uma variável chamada previsoes para realizar a previsão dos dados de teste (X_teste) O resultado da rede deve ser um valor entre 0 e 1. Porque isso acontece?

In [43]:
previsoes = rede_neural.predict(X_teste)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [44]:
# Visualizar as previsões
print("Primeiras 10 previsões:")
print(previsoes[:10])
print(f"\nValor mínimo: {previsoes.min():.4f}")
print(f"Valor máximo: {previsoes.max():.4f}")
print(f"Valor médio: {previsoes.mean():.4f}")
print(f"Todas as previsões estão entre 0 e 1: {(previsoes >= 0).all() and (previsoes <= 1).all()}")

Primeiras 10 previsões:
[[0.99866575]
 [0.9999676 ]
 [0.99718267]
 [0.99677867]
 [0.9990554 ]
 [0.996568  ]
 [0.99997956]
 [0.9996904 ]
 [0.9945668 ]
 [0.99772656]]

Valor mínimo: 0.0000
Valor máximo: 1.0000
Valor médio: 0.9817
Todas as previsões estão entre 0 e 1: True


**Resposta 9:** O resultado está entre 0 e 1 porque a camada de saída usa a função de ativação sigmoid, que matematicamente transforma qualquer valor de entrada em um valor entre 0 e 1. Isso representa a probabilidade da amostra pertencer à classe positiva (maligno = 1). Valores próximos a 1 indicam alta probabilidade de ser maligno, e valores próximos a 0 indicam alta probabilidade de ser benigno.

10) Converta o resultado que está em probabilidade para valores binários com um limiar (treshold) de 0.5.

In [45]:
previsoes = previsoes > 0.5

In [46]:
previsoes

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [

9) Explique o resultado da RNA.

In [47]:
# Verificar a distribuição REAL dos dados
print("Distribuição REAL no dataset completo:")
print(f"Benignos (0): {(y == 0).sum()} ({(y == 0).mean():.2%})")
print(f"Malignos (1): {(y == 1).sum()} ({(y == 1).mean():.2%})")

print("\nDistribuição no conjunto de TESTE:")
print(f"Benignos (0): {(y_teste == 0).sum()} ({(y_teste == 0).mean():.2%})")
print(f"Malignos (1): {(y_teste == 1).sum()} ({(y_teste == 1).mean():.2%})")

# Visualizar as previsões binárias
print("Primeiras 10 previsões:")
print(previsoes[:10].flatten())  # Mostra True/False

# Estatísticas para dados binários
previsoes_num = previsoes.astype(int).flatten()
print(f"\nTotal de predições classe 0 (benigno): {(previsoes_num == 0).sum()}")
print(f"Total de predições classe 1 (maligno): {(previsoes_num == 1).sum()}")
print(f"\nPorcentagem real de malignos no teste: {(y_teste == 1).mean():.2%}")
print(f"Proporção de casos malignos: {previsoes_num.mean():.2%}")


Distribuição REAL no dataset completo:
Benignos (0): 213 (37.43%)
Malignos (1): 356 (62.57%)

Distribuição no conjunto de TESTE:
Benignos (0): 61 (42.66%)
Malignos (1): 82 (57.34%)
Primeiras 10 previsões:
[ True  True  True  True  True  True  True  True  True  True]

Total de predições classe 0 (benigno): 2
Total de predições classe 1 (maligno): 141

Porcentagem real de malignos no teste: 57.34%
Proporção de casos malignos: 98.60%


**Resposta:** O método `evaluate()` retorna dois valores: a perda (loss) e a acurácia binária. A perda indica o quão distantes estão as previsões dos valores reais (quanto menor, melhor). A acurácia binária mostra a porcentagem de classificações corretas no conjunto de teste. Esses resultados permitem avaliar o desempenho do modelo em dados não vistos durante o treinamento.

## Camadas e Otimização da RNA

10) Adicione mais uma camada para oculta densa com 16 neurônio com a função de ativação relu e inicialize os pesos de utilize o Random uniform initializer. Qual o total de parâmetros da RNA agora?

In [48]:
# Recriar a rede neural com duas camadas ocultas
rede_neural = Sequential([
    tf.keras.layers.InputLayer(shape=(30,)),
    tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
    tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),  # Nova camada
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

In [49]:
rede_neural.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 785 (3.07 KB)

 Trainable params: 785 (3.07 KB)

 Non-trainable params: 0 (0.00 B)

11) Abaixo são adicionados os parâmetros do otimizador, que são a taxa de aprendizado e o clipvalue. O que eles fazem?

In [50]:
otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

**Resposta 11:**
- **learning_rate (taxa de aprendizado)**: Controla o tamanho dos passos que o otimizador dá ao ajustar os pesos. Um valor de 0.001 significa passos pequenos e cuidadosos, evitando oscilações mas permitindo convergência gradual.
- **clipvalue**: Limita os gradientes ao intervalo [-0.5, 0.5]. Isso previne o problema de "gradientes explosivos" (exploding gradients), onde gradientes muito grandes podem desestabilizar o treinamento.

12) Teste novamente a RNA. Aumentar a quantidade de camadas melhorou ou piorou os resultados? Explique o que aconteceu com a RNA e porque.

In [51]:
rede_neural.fit(X_treinamento, y_treinamento, batch_size = 10, epochs = 100)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - binary_accuracy: 0.5235 - loss: 1.1787
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5681 - loss: 1.2155 
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5986 - loss: 0.8959 
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5869 - loss: 0.9237 
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5869 - loss: 0.9587
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5986 - loss: 0.8678
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - binary_accuracy: 0.6103 - loss: 0.8074
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.5962 - loss: 0.8349 
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6127 - loss: 1.0404 
Epoch 10/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6479 - loss: 0.7846 
Epoch 11/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0

In [52]:
previsoes = rede_neural.predict(X_teste)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [53]:
rede_neural.evaluate(X_teste, y_teste)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - binary_accuracy: 0.5105 - loss: 0.8685  


[0.8685498237609863, 0.5104895234107971]

In [54]:
# Verificar a distribuição REAL dos dados
print("Distribuição REAL no dataset completo:")
print(f"Benignos (0): {(y == 0).sum()} ({(y == 0).mean():.2%})")
print(f"Malignos (1): {(y == 1).sum()} ({(y == 1).mean():.2%})")

print("\nDistribuição no conjunto de TESTE:")
print(f"Benignos (0): {(y_teste == 0).sum()} ({(y_teste == 0).mean():.2%})")
print(f"Malignos (1): {(y_teste == 1).sum()} ({(y_teste == 1).mean():.2%})")

# Visualizar as probabilidades ANTES da conversão
print("\nPrimeiras 10 probabilidades:")
print(previsoes[:10].flatten())

# CONVERTER para binário (0 ou 1) usando threshold de 0.5
previsoes_binarias = (previsoes > 0.5).astype(int).flatten()

# Visualizar as previsões APÓS conversão
print("\nPrimeiras 10 previsões binárias:")
print(previsoes_binarias[:10])

# Estatísticas das PREDIÇÕES
print(f"\nTotal de predições classe 0 (benigno): {(previsoes_binarias == 0).sum()}")
print(f"Total de predições classe 1 (maligno): {(previsoes_binarias == 1).sum()}")
print(f"Proporção de casos malignos preditos: {previsoes_binarias.mean():.2%}")

# Comparar com a realidade
print(f"\nPorcentagem real de malignos no teste: {(y_teste == 1).mean():.2%}")
print(f"Porcentagem predita de malignos: {previsoes_binarias.mean():.2%}")


Distribuição REAL no dataset completo:
Benignos (0): 213 (37.43%)
Malignos (1): 356 (62.57%)

Distribuição no conjunto de TESTE:
Benignos (0): 61 (42.66%)
Malignos (1): 82 (57.34%)

Primeiras 10 probabilidades:
[0.67117894 0.46940288 0.46940288 0.46940288 0.64579904 0.7384499
 0.86413693 0.46940288 0.6003682  0.60030925]

Primeiras 10 previsões binárias:
[1 0 0 0 1 1 1 0 1 1]

Total de predições classe 0 (benigno): 47
Total de predições classe 1 (maligno): 96
Proporção de casos malignos preditos: 67.13%

Porcentagem real de malignos no teste: 57.34%
Porcentagem predita de malignos: 67.13%


## K-Fold Cross Validation

13) Assista o vídeo https://youtu.be/RczbeFs_WbQ?si=f1Yu4LZaGDfZZpbZ para compreender melhor sobre como funciona o k-fold cross validation. Explique como esta rede foi configurada e como é possível chegar no resultado dela. O que é necessário fazer?

**Resposta 13:** O K-Fold Cross Validation divide os dados em K partes (folds). Para cada iteração, uma parte é usada como teste e as demais como treino, repetindo K vezes. A rede é configurada através da função `criar_rede()` que define a arquitetura, e o `cross_val_score` treina e avalia K vezes (cv=10), retornando a acurácia de cada fold. Isso permite uma avaliação mais robusta do modelo.

14) Calcule também o Desvio Padrão dos resultados para avaliar o modelo. O que é possível concluir com esse resultado?

**Resposta 14:** O desvio padrão mede a variabilidade dos resultados entre os diferentes folds. Um desvio padrão baixo indica que o modelo tem desempenho consistente e estável. Um desvio alto sugere que o modelo é sensível à divisão dos dados, podendo indicar overfitting ou instabilidade no treinamento.



In [55]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as k

In [56]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

In [57]:
rede_neural = KerasClassifier(model = criar_rede, epochs = 100, batch_size = 10)

In [ ]:
resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 10, scoring = 'accuracy')

Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - binary_accuracy: 0.5840 - loss: 1.0356
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5664 - loss: 0.8309
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5859 - loss: 0.8028
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.5645 - loss: 0.8456
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6152 - loss: 0.7423
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5879 - loss: 0.7529
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6094 - loss: 0.7134
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5957 - loss: 0.8155
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6094 - loss: 0.7901
Epoch 10/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6250 - loss: 0.7085
Epoch 11/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6172 -

In [ ]:
resultados

In [ ]:
# Calcular média e desvio padrão dos resultados
media = resultados.mean()
desvio_padrao = resultados.std()
print(f"Acurácia Média: {media:.4f}")
print(f"Desvio Padrão: {desvio_padrao:.4f}")

## Overfitting e Dropout

14) Como vimos, o modelo anterior sofreu overfitting. Para reduzir esse problema, aplicamos a técnica de regularização conhecida como dropout (artigo oficial: https://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf e explicação em português: https://www.deeplearningbook.com.br/como-funciona-o-dropout/ ). Aplique o dropout de 20% na primeira e segunda camada oculta. O que acontece com os resultados? E o Desvio Padrão?

In [ ]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dropout(rate=0.2),  # Dropout de 20% após a primeira camada oculta
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dropout(rate=0.2),  # Dropout de 20% após a segunda camada oculta
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

In [ ]:
rede_neural = KerasClassifier(model = criar_rede, epochs = 100, batch_size = 10)

In [ ]:
resultados = cross_val_score(estimator = rede_neural, X = X, y = y, cv = 10, scoring = 'accuracy')

In [ ]:
resultados

In [ ]:
# Calcular média e desvio padrão dos resultados com dropout
media_dropout = resultados.mean()
desvio_padrao_dropout = resultados.std()
print(f"Acurácia Média (com Dropout): {media_dropout:.4f}")
print(f"Desvio Padrão (com Dropout): {desvio_padrao_dropout:.4f}")

**Resposta 14:** O dropout ajuda a reduzir o overfitting ao "desligar" aleatoriamente 20% dos neurônios durante o treinamento, forçando a rede a aprender representações mais robustas. Geralmente, isso resulta em uma acurácia de treino ligeiramente menor, mas uma acurácia de teste melhor (melhor generalização). O desvio padrão tende a diminuir, indicando que o modelo é mais consistente entre diferentes folds.

## Tuning dos Hiperparâmetros (técnicas de ajuste)

Hyperparameter Tuning (Ajuste de Hiperparâmetros) é o processo geral de encontrar os melhores valores para os hiperparâmetros de um modelo de machine learning.
Exemplos de hiperparâmetros:

- Taxa de aprendizado (learning rate)

- Número de árvores em um RandomForest

- Número de camadas ou neurônios em uma rede neural

- Tipo de kernel em um SVM

O tuning pode ser feito por diferentes técnicas, como:

- Grid Search

- Random Search

- Bayesian Optimization, etc.

15) Descreva como a RNA foi configurada para fazer o processo de tuning.

**Resposta 15:** A RNA foi configurada de forma parametrizada através da função `criar_rede()`, que agora aceita múltiplos parâmetros (optimizer, loss, kernel_initializer, activation, neurons). Esses parâmetros são definidos no dicionário `parametros` com diferentes valores a serem testados. O GridSearchCV testa todas as combinações possíveis desses parâmetros, treinando e avaliando o modelo para cada combinação, retornando os melhores parâmetros encontrados.

In [ ]:
from sklearn.model_selection import GridSearchCV

def criar_rede(optimizer, loss, kernel_initializer, activation, neurons):
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  rede_neural.compile(optimizer = optimizer, loss = loss, metrics = ['binary_accuracy'])
  return rede_neural

In [ ]:
rede_neural = KerasClassifier(model = criar_rede)

In [ ]:
parametros = {
    'batch_size': [10, 30],
    'epochs': [50, 100],
    'model__optimizer': ['adam', 'sgd'],
    'model__loss': ['binary_crossentropy', 'hinge'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__activation': ['relu', 'tanh'],
    'model__neurons': [16, 8]
}

In [ ]:
#caso tiver problema com a configuração anterior, testar essa
parametros = {
    'batch_size': [10, 30],
    'epochs': [50],
    'model__optimizer': ['adam'],
    'model__loss': ['binary_crossentropy'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__activation': ['relu'],
    'model__neurons': [16]
}

In [ ]:
parametros

In [ ]:
grid_search = GridSearchCV(estimator = rede_neural, param_grid = parametros,
                          scoring = 'accuracy', cv = 5)

In [ ]:
grid_search = grid_search.fit(X, y)

In [ ]:
print(grid_search)

In [ ]:
melhores_parametros = grid_search.best_params_
print(melhores_parametros)

In [ ]:
melhor_precisao = grid_search.best_score_
print(melhor_precisao)

16) É possível melhorar ainda mais a curácia da RNA? Como?

**Resposta 16:** Sim, é possível melhorar a acurácia através de várias técnicas:
- **Normalização/Padronização dos dados**: Usar StandardScaler ou MinMaxScaler para escalar as features
- **Aumentar épocas**: Treinar por mais tempo, com early stopping para evitar overfitting
- **Ajustar arquitetura**: Testar diferentes números de camadas e neurônios
- **Regularização L1/L2**: Adicionar penalização aos pesos
- **Batch Normalization**: Normalizar as saídas entre camadas
- **Ajustar learning rate**: Usar learning rate schedulers ou valores diferentes
- **Aumentar dados**: Data augmentation (se aplicável) ou coletar mais amostras
- **Ensemble methods**: Combinar múltiplos modelos

#Para salvar o seu modelo de RNA

In [ ]:
classificador.save('classificador_breast.keras')

In [ ]:
classificador_novo = tf.keras.models.load_model('/content/classificador_breast.keras')

In [ ]:
classificador_novo.summary()